In [1]:
import pandas as pd
import plotly.express as px

# from google.colab import drive
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset
import torch.nn as nn
import torch

from tqdm import tqdm_notebook as tqdm
import logging


/Users/caioseda/opt/miniconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_STATE = 2023
BATCH_SIZE = 2048
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Selected device is {device}')

Selected device is cuda


In [ ]:
49836

In [3]:
cd_list = pd.read_csv('/kaggle/input/tce-nota-empenho/cd_list.csv', sep=';')
display(cd_list.head())
cd_list.shape

,cod,titulo,desc,tituloTCE,freq,level,parent,root,child
0,30000000,DESPESAS CORRENTES,CLASSIFICAM SE NESTA CATEGORIA TODAS AS DESPES...,DESPESAS CORRENTES,0,1 - Categoria,30000000,root,3
1,31000000,PESSOAL E ENCARGOS SOCIAIS,DESPESAS DE NATUREZA REMUNERATORIA DECORRENTES...,PESSOAL E ENCARGOS SOCIAIS,0,2 - Grupo,30000000,3,31
2,31200000,TRANSFERENCIAS A UNIAO,0,TRANSFERENCIAS A UNIAO,0,3 - Modalidade,31000000,31,3120
3,31209600,RESSARCIMENTO DE DESPESAS DE PESSOAL REQUISITADO,RESSARCIMENTO DE DESPESAS DE PESSOAL REQUISITADO,RESSARCIMENTO DE DESPESAS DE PESSOAL REQUISITADO,4,4 - Elemento,31200000,3120,312096
4,31300000,TRANSFERENCIAS A ESTADOS E AO DISTRITO FEDERAL,0,TRANSFERENCIAS A ESTADOS E AO DISTRITO FEDERAL,0,3 - Modalidade,31000000,31,3130


(1774, 9)

In [ ]:
cd_list

In [4]:
tce_ml = pd.read_csv('/kaggle/input/tce-nota-empenho/tce_ML.csv', sep=';')
display(tce_ml.head())
tce_ml.shape

/tmp/ipykernel_28/1168473016.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  tce_ml = pd.read_csv('/kaggle/input/tce-nota-empenho/tce_ML.csv', sep=';')


,df_index,Ano,CdFonteTCE,CPFCNPJCredor,Credor,DEFonteTCE,DEPrograma,ElemDespesaTCE,Funcao,Historico,...,Tp_Empenho,Unidade,CGProgTrab,Cod_Elem,CG,CGtitulo,CGtitTCE,CGfreq,CGroot,CGchild
0,227018,2021,0,191,BANCO DO BRASIL SA,RECURSOS ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,ADMINISTRACAO,CHAMAMENTO PUBLICO 01 2020 SFI PROC 2019021...,...,ESTIMATIVO,PREFEITURA ANGRA DOS REIS,33903999,33904000.0,33903999,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,33616,339039,33903999
1,227039,2021,0,85822000112,ESPECIFARMA COM DE MEDICAMENTOS E PROD,RECURSOS ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,SENTENCAS JUDICIAIS,ADMINISTRACAO,AQUISICAO DE MEDICAMENTOS PARA O CUMPRIMENTO D...,...,ORDINARIO,PREFEITURA ANGRA DOS REIS,33909101,33909100.0,33909101,SENTENCAS JUDICIAIS,SENTENCAS JUDICIAIS,420,339091,33909101
2,227071,2021,77,304942000163,SALE SERVICE INDUSTRICA E COMERCIO E SERVICOS,RECURSOS VINCULADOS AO TRANSITO,MOBILIDADE URBANA E TRANSITO,MATERIAL DE CONSUMO,ADMINISTRACAO,AQUISICAO DE TINTAS E SOLVENTE PARA SINALIZACA...,...,ORDINARIO,PREFEITURA ANGRA DOS REIS,33903016,33903016.0,33903016,MATERIAIS DE CONSUMO PARA ACAMPAMENTO E CAMPANHA,MATERIAL DE CONSUMO,532,339030,33903016
3,227077,2018,0,360305000104,CAIXA ECONOMICA FEDERAL,ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,CONTRIBUICAO PARA O FGTS,SAUDE,TRATA SE DE DESPESA COM PAGAMENTO DE FGTS DOS ...,...,ORDINARIO,PREFEITURA ANGRA DOS REIS,31901301,31901300.0,31901301,FGTS,OUTROS ENCARGOS PATRONAIS,7,319013,31901301
4,227120,2021,0,360305000104,CAIXA ECONOMICA FEDERAL,RECURSOS ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,ADMINISTRACAO,CHAMAMENTO PUBLICO 01 2020 SFI PROC 2019021...,...,ESTIMATIVO,PREFEITURA ANGRA DOS REIS,33903999,33904000.0,33903999,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,33616,339039,33903999


(209173, 27)

In [5]:
tce_ml.shape

(209173, 27)

In [6]:
tce_ml.columns

Index(['df_index', 'Ano', 'CdFonteTCE', 'CPFCNPJCredor', 'Credor',
       'DEFonteTCE', 'DEPrograma', 'ElemDespesaTCE', 'Funcao', 'Historico',
       'IdContrato', 'IdPrograma', 'IdSubFuncao', 'IdUnid', 'IdOrgao',
       'NrFonte', 'SubFuncao', 'Tp_Empenho', 'Unidade', 'CGProgTrab',
       'Cod_Elem', 'CG', 'CGtitulo', 'CGtitTCE', 'CGfreq', 'CGroot',
       'CGchild'],
      dtype='object')

In [7]:
N = 10

frequencia_cg = tce_ml.CG.value_counts(normalize=True)
frequencia_cg_top_N = frequencia_cg.iloc[:N]
frequencia_cg_top_N_outros = pd.Series(frequencia_cg.iloc[N:].sum(), index=['outros'])

frequencia_cg_top_N = frequencia_cg_top_N.append(frequencia_cg_top_N_outros)
frequencia_cg_top_N = frequencia_cg_top_N.reset_index(name='freq').rename(columns={'index':'cg'})
frequencia_cg_top_N.cg = frequencia_cg_top_N.cg.astype(str)

fig = px.bar(y=frequencia_cg_top_N.freq, x=frequencia_cg_top_N.cg)
fig.update_layout(title='Histograma CG', yaxis_tickformat='p', xaxis_title='CG class', yaxis_title='%')

/tmp/ipykernel_28/2584687476.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencia_cg_top_N = frequencia_cg_top_N.append(frequencia_cg_top_N_outros)


In [8]:
tce_ml

,df_index,Ano,CdFonteTCE,CPFCNPJCredor,Credor,DEFonteTCE,DEPrograma,ElemDespesaTCE,Funcao,Historico,...,Tp_Empenho,Unidade,CGProgTrab,Cod_Elem,CG,CGtitulo,CGtitTCE,CGfreq,CGroot,CGchild
0,227018,2021,0,191,BANCO DO BRASIL SA,RECURSOS ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,ADMINISTRACAO,CHAMAMENTO PUBLICO 01 2020 SFI PROC 2019021...,...,ESTIMATIVO,PREFEITURA ANGRA DOS REIS,33903999,33904000.0,33903999,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,33616,339039,33903999
1,227039,2021,0,85822000112,ESPECIFARMA COM DE MEDICAMENTOS E PROD,RECURSOS ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,SENTENCAS JUDICIAIS,ADMINISTRACAO,AQUISICAO DE MEDICAMENTOS PARA O CUMPRIMENTO D...,...,ORDINARIO,PREFEITURA ANGRA DOS REIS,33909101,33909100.0,33909101,SENTENCAS JUDICIAIS,SENTENCAS JUDICIAIS,420,339091,33909101
2,227071,2021,77,304942000163,SALE SERVICE INDUSTRICA E COMERCIO E SERVICOS,RECURSOS VINCULADOS AO TRANSITO,MOBILIDADE URBANA E TRANSITO,MATERIAL DE CONSUMO,ADMINISTRACAO,AQUISICAO DE TINTAS E SOLVENTE PARA SINALIZACA...,...,ORDINARIO,PREFEITURA ANGRA DOS REIS,33903016,33903016.0,33903016,MATERIAIS DE CONSUMO PARA ACAMPAMENTO E CAMPANHA,MATERIAL DE CONSUMO,532,339030,33903016
3,227077,2018,0,360305000104,CAIXA ECONOMICA FEDERAL,ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,CONTRIBUICAO PARA O FGTS,SAUDE,TRATA SE DE DESPESA COM PAGAMENTO DE FGTS DOS ...,...,ORDINARIO,PREFEITURA ANGRA DOS REIS,31901301,31901300.0,31901301,FGTS,OUTROS ENCARGOS PATRONAIS,7,319013,31901301
4,227120,2021,0,360305000104,CAIXA ECONOMICA FEDERAL,RECURSOS ORDINARIOS,GESTAO MANUTENCAO E SERVICOS DO MUNICIPIO,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,ADMINISTRACAO,CHAMAMENTO PUBLICO 01 2020 SFI PROC 2019021...,...,ESTIMATIVO,PREFEITURA ANGRA DOS REIS,33903999,33904000.0,33903999,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,OUTROS SERVICOS DE TERCEIROS PESSOA JURIDICA,33616,339039,33903999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209168,1838454,2021,15,5400781000125,FUNDO DE PREVIDENCIA DO MUNICIPAL DE ITALVA,TRANSFERENCIAS DO FUNDEB 70,FUNDEB,CONTRIBUICOES PREVIDENCIARIAS RPPS PESSOAL...,EDUCACAO,DESPESAS COM OBIGACOES PATRONAIS E RETENCOES C...,...,ESTIMATIVO,FUN MUNI DE EDUCACAO DE ITALVA,31911303,31911304.0,31911303,CONTRIBUICOES PATRONAIS PESSOAL CIVIL PLANO...,CONTRIBUICOES PREVIDENCIARIAS RPPS PESSOAL...,8,319113,31911303
209169,1838455,2021,15,6083793000136,MERITI PREVI,TRANSFERENCIAS DO FUNDEB 70,FUNDEB,OUTROS OBRIGACOES PATRONAIS,EDUCACAO,DESPESA REFERENTE A OBRIGACOES PATRONAIS A SER...,...,ESTIMATIVO,FUN MUNI DE EDUCACAO DE ITALVA,31901399,31901400.0,31901399,CONTRIBUICAO PARA O REGIME GERAL DE PREVIDENCI...,CONTRIBUICAO PARA O REGIME GERAL DE PREVIDENCI...,8,319013,31901399
209170,1838522,2021,0,29979036024830,INSTITUTO NACIONAL DO SEGURO SOCIAL,RECURSOS ORDINARIOS,EDUCACAO,CONTRIBUICAO PARA O REGIME GERAL DE PREVIDENCI...,EDUCACAO,DESPESAS COM OBIGACOES PATRONAIS E RETENCOES C...,...,ESTIMATIVO,FUN MUNI DE EDUCACAO DE ITALVA,31901302,31901304.0,31901302,INSS,CONTRIBUICAO PARA O REGIME PROPRIO DE PREVIDEN...,576,319013,31901302
209171,1838523,2021,15,29979036024830,INSTITUTO NACIONAL DO SEGURO SOCIAL,TRANSFERENCIAS DO FUNDEB 70,FUNDEB,CONTRIBUICAO PARA O REGIME GERAL DE PREVIDENCI...,EDUCACAO,DESPESAS COM OBIGACOES PATRONAIS E RETENCOES C...,...,ESTIMATIVO,FUN MUNI DE EDUCACAO DE ITALVA,31901302,31901304.0,31901302,INSS,CONTRIBUICAO PARA O REGIME PROPRIO DE PREVIDEN...,576,319013,31901302


## 1 - Extraindo Word Embeddings da coluna Historico usando BERTimbau

Referências:
- http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
- https://www.youtube.com/watch?v=kRyOAapLpIo
- https://huggingface.co/neuralmind/bert-base-portuguese-cased

### 1.1 - Importando modelos

In [9]:
MAX_LENGTH = 512

In [10]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device)

# DataParallel enables the use of Multiple GPUs!
model = nn.DataParallel(model)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning:

unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning:

file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tensorflow13GcsFileSystemE']



Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 1.2 - Exemplo de tokenização

In [11]:
exemplo = tce_ml.sample(1, random_state=RANDOM_STATE).iloc[0]
exemplo['Historico']

'CONTRATACAO DE EMPRESA ESPECIALIZADA PARA REALIZACAO DE SERVICOS TECNICOS DE ASSESSORIA E CONSULTORIA DE CALCULO ATUARIAL EM ATENDIMENTO AO FUNPREV RC  COM A PRESTACAO DE TODOS OS SERVICOS NECESSARIOS PARA FINS DE CUMPRIMENTO DA LEI NO 9717 98 E CONS'

In [12]:
tokenizer.encode(
    exemplo['Historico'],
    add_special_tokens=True,
    padding=True,
    truncation=True,
    max_length=MAX_LENGTH
)

[101,
 16273,
 22321,
 5650,
 5118,
 4529,
 22317,
 10836,
 192,
 11881,
 7286,
 5903,
 20697,
 5476,
 5234,
 22327,
 22323,
 22351,
 11836,
 22301,
 18868,
 5650,
 257,
 16379,
 22327,
 22323,
 22351,
 7436,
 22301,
 22317,
 10836,
 14128,
 22322,
 19893,
 5218,
 22308,
 267,
 5476,
 22320,
 6162,
 6072,
 10836,
 21468,
 13925,
 6236,
 8718,
 5234,
 192,
 16273,
 22308,
 18199,
 15349,
 21748,
 22301,
 10836,
 17715,
 14752,
 18199,
 22317,
 20415,
 3168,
 21748,
 9369,
 192,
 22311,
 20415,
 12547,
 18822,
 7463,
 8427,
 22317,
 177,
 22317,
 263,
 11964,
 8519,
 22309,
 22339,
 257,
 22304,
 6213,
 22311,
 177,
 11635,
 3341,
 5118,
 4529,
 22317,
 10836,
 267,
 22317,
 18504,
 22308,
 14405,
 14128,
 22322,
 19893,
 5218,
 22308,
 248,
 5476,
 3341,
 5903,
 21748,
 6072,
 18868,
 5650,
 5769,
 22320,
 22308,
 10836,
 187,
 22341,
 11881,
 21748,
 7463,
 8427,
 22317,
 250,
 22301,
 276,
 18469,
 20660,
 18085,
 8307,
 15405,
 192,
 16273,
 22308,
 102]

![image.png](attachment:image.png)

### 1.3 - Tokenizando a coluna Historico

In [13]:
# Gera tokens e attention_mask (padding_mask) para cada linha no dataset em PyTorch Tensors
tokens_obj = tokenizer.batch_encode_plus( 
    tce_ml['Historico'].values.tolist(),
    return_tensors='pt', 
    add_special_tokens=True,
    padding=True,
    truncation=True,
    max_length=MAX_LENGTH
)
tokens, _, attention_mask = tokens_obj.values()

In [14]:

class HistoricoDataset(Dataset):
  def __init__(self, tokens, attention_mask, device=device):
    self.tokens = tokens.to(device)
    self.attention_mask = attention_mask.to(device)
    self.length = len(tokens)

  def __len__(self):
    return self.length

  def __getitem__(self, idx):
    return self.tokens[idx ,:], self.attention_mask[idx ,:]

In [15]:
historico_dataset = HistoricoDataset(tokens, attention_mask, device=device)
historico_dataloader = torch.utils.data.DataLoader(historico_dataset, batch_size=BATCH_SIZE)

In [16]:
number_of_batchs = len(tokens)/BATCH_SIZE
print(f'Estimated number of batches is {int(number_of_batchs)}')

Estimated number of batches is 102


In [ ]:
hidden_states_list = []
with torch.no_grad():
  for batch_number, (batch_tokens, batch_attention_mask) in tqdm(enumerate(historico_dataloader)):
    hidden_states_batch = model(batch_tokens, attention_mask=batch_attention_mask)
    
    # Created a Matrix of size:
    # (n_observations, max_n_of_tokens_in_sentence, n_hidden_states_in_bertimbau)
    # For classification task, we just want the hidden states for the first token
    hidden_states_batch = hidden_states_batch[0][:,0,:].cpu().numpy()
    hidden_states_list.append(hidden_states_batch)

/tmp/ipykernel_28/3492724167.py:3: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



0it [00:00, ?it/s]

In [18]:
import numpy as np
hidden_states = np.concatenate(hidden_states_list)
hidden_states = pd.DataFrame(hidden_states)
hidden_states.to_csv('historico-word-embeddings-cls-state.csv', index=False)

In [23]:
hidden_states

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.374859,-0.355990,0.879074,-0.276560,0.890596,0.378356,0.287249,0.156563,0.514426,0.319978,...,-0.201854,-0.462453,-0.653172,-0.121306,0.347493,0.287559,-0.113759,0.342992,-0.129540,0.349259
1,0.265291,0.068703,0.750341,-0.060533,0.954906,0.573185,0.270869,0.163694,0.153029,0.427151,...,-0.202540,-0.730261,-0.407751,0.037137,0.645356,0.484363,-0.199481,0.239201,-0.393485,0.286100
2,0.290201,0.049243,0.756773,-0.080436,1.047454,0.335303,0.526496,0.036686,0.301992,0.512087,...,-0.286225,-0.606716,-0.433432,0.120021,0.663979,0.762440,-0.308434,0.527319,-0.317191,0.241087
3,0.474922,0.074208,1.034273,-0.030208,0.773727,0.299223,0.333751,0.073844,0.554678,0.487423,...,0.192715,-0.379011,-0.155515,0.170225,0.227142,0.636074,-0.282616,0.559989,-0.265717,-0.144155
4,0.374859,-0.355990,0.879074,-0.276560,0.890596,0.378356,0.287249,0.156563,0.514426,0.319978,...,-0.201854,-0.462453,-0.653172,-0.121306,0.347493,0.287559,-0.113759,0.342992,-0.129540,0.349259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209168,0.377154,-0.470555,0.702630,-0.111019,0.691515,0.172321,-0.307324,0.575841,0.457927,0.400745,...,-0.327506,-0.273192,-0.600196,0.182716,0.434408,0.292839,-0.104905,0.141586,0.004000,0.118608
209169,0.511300,-0.121021,0.897727,-0.209290,0.872651,0.491595,0.249380,0.240569,0.195053,0.328241,...,-0.357738,-0.409742,-0.580369,0.071413,0.721721,0.628019,-0.007405,0.036193,-0.284393,0.156770
209170,0.282660,-0.451844,0.781164,-0.247282,0.800144,0.172179,-0.091117,0.451921,0.112741,0.447550,...,-0.374027,-0.513144,-0.577108,-0.009710,0.862981,0.533980,-0.093446,0.061105,-0.073697,-0.021309
209171,0.329787,-0.525615,0.764593,-0.180871,0.818519,0.215418,-0.236186,0.569049,0.364163,0.340847,...,-0.291371,-0.408480,-0.601665,0.288301,0.590432,0.412432,-0.149814,0.191954,-0.042346,0.045932
